In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import folium
import csv

# Recover Ripple Graph

In [2]:
def recover_links_ases(filename):
    links = set()
    ases = set()
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            l = tuple(row)
            links.add(l)
            ases.add(l[0])
            ases.add(l[1])
    return ases,links

In [3]:
ripple_ases, ripple_links = recover_links_ases('Ripple/gateway_links.csv')

In [4]:
ripple_graph = nx.Graph()
ripple_graph.add_nodes_from(ripple_ases)

In [5]:
print('The Ripple graph contains {} nodes'.format(len(ripple_graph.nodes)))

The Ripple graph contains 15 nodes


# Recover Caida Graph

In [6]:
data = pd.read_csv('Caida/20190301.as-rel2.txt',delimiter='\n', comment='#', header=None, encoding='ISO-8859-1')

data['provider_peer'], data['customer_peer'], data['relation_type'], data['source'] = data[0].str.split('|').str
data['relation_type']  = data['relation_type'].apply(lambda x: 'provider-customer' if x == '-1' else 'peer-peer')
links = data[['provider_peer','customer_peer','relation_type']]

caida_ases = set(links['provider_peer'].values)
caida_ases.update(links['customer_peer'].values)

links['link'] = list(zip(links['provider_peer'], links['customer_peer']))
links = links[['link','relation_type']]
caida_links = {}
for index,row in links.iterrows():
    (s,t) = row['link']
    rel = row['relation_type']
    if rel == 'provider-customer':
        caida_links[tuple([s,t])] = rel
        caida_links[tuple([t,s])] = 'customer-provider'
    else: 
        if rel == 'peer-peer':
            caida_links[tuple([s,t])] = rel
            caida_links[tuple([t,s])] = 'peer-peer'
        else:
            print('Error')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [7]:
caida_graph = nx.Graph()
caida_graph.add_nodes_from(caida_ases)
for k,v in caida_links.items():
    caida_graph.add_edge(k[0],k[1])

In [8]:
print('The Caida graph contains {} nodes and {} edges'.format(len(caida_graph.nodes),len(caida_graph.edges)))

The Caida graph contains 64167 nodes and 503651 edges


# Add intermediary nodes to Ripple Graph

In [9]:
def shortest_paths(s,t,G):
    actual_paths = []
    paths = nx.all_shortest_paths(G,s,t)
    for path in paths:
        actual_path = []
        for i in range(len(path)-1):
            l = tuple([path[i],path[i+1]])
            actual_path.append(l)
        actual_paths.append(actual_path)
    return actual_paths

In [10]:
for s in ripple_ases:
    for t in ripple_ases:
        paths = nx.all_shortest_paths(caida_graph,s,t)
        for p in paths:
            complete_path = [s] + p
            for i in range(len(complete_path)-1):
                ripple_graph.add_edge(complete_path[i],complete_path[i+1])

In [11]:
print('The Ripple graph contains {} nodes and {} edges'.format(len(ripple_graph.nodes),len(ripple_graph.edges)))

The Ripple graph contains 324 nodes and 1299 edges


In [12]:
nx.set_edge_attributes(ripple_graph, 0, 'weight')

# Replay transactions 

In [13]:
transactions = pd.read_csv('Ripple/transactions_processed.csv', dtype={'sender': str, 'receiver': str, 'amount': np.float64} )
transactions = transactions[['sender','receiver','amount']]

In [14]:
def replay_transactions(transactions, corrupted_graph, complete_graph):
    amount_ok = 0
    amount_lost = 0
    amount_rerouted = 0
    for index, row in transactions.iterrows():
        sender = row['sender']
        receiver = row['receiver']

        try :
            amount = row['amount']
            ## For the moment, use only the best path
            path_corrupted = nx.shortest_path(corrupted_graph,sender,receiver)
            path_ok = nx.shortest_path(complete_graph,sender,receiver)
            if (path_corrupted == path_ok):
                amount_ok += amount
            else:
                amount_rerouted += amount
        except:
            #print('No path between {} and {}'.format(sender,receiver))
            amount_lost += amount
    return amount_ok, amount_lost, amount_rerouted

In [15]:
def analyse_graph(graph, ripple_ases):
    for d in graph.degree(weight='weight'):
        if d[0] in ripple_ases:
            print('The following node is a Ripple node:')
        print('AS: {} - Money: {}\n'.format(d[0],d[1]))

In [16]:
## Very bad TODO
amount_ok, amount_lost, amount_rerouted = replay_transactions(transactions, ripple_graph, ripple_graph)
print('Amount of money used in the graph : {}'.format(amount_ok))
#analyse_graph(ripple_graph, ripple_ases)

Amount of money used in the graph : 26624832.40333599


# What if we remove one node?

## AS containing a Ripple gateway

In [17]:
for c in ripple_ases:
    corrupted_graph = ripple_graph.copy()
    corrupted_graph.remove_node(c)
    nx.set_edge_attributes(corrupted_graph, 0, 'weight')
    print('What if {} is corrupted?'.format(c))
    amount_ok, amount_lost, amount_rerouted = replay_transactions(transactions, corrupted_graph, ripple_graph)
    total = amount_ok + amount_lost + amount_rerouted
    percent_ok = amount_ok / total
    percent_ko = amount_lost / total
    percent_rerouted = amount_rerouted / total
    print('Money OK : {}, {:.0%} - Money lost : {}, {:.0%} - Money rerouted : {}, {:.0%} \n'.format(amount_ok, percent_ok, amount_lost, percent_ko, amount_rerouted, percent_rerouted))
    #analyse_graph(ripple_graph, ripple_ases)

What if 37907 is corrupted?
Money OK : 26624832.403263155, 100% - Money lost : 7.283394449113598e-05, 0% - Money rerouted : 0, 0% 

What if 9371 is corrupted?
Money OK : 26620162.892369736, 100% - Money lost : 4669.510966250023, 0% - Money rerouted : 0, 0% 

What if 57127 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 38895 is corrupted?
Money OK : 24813575.720647603, 93% - Money lost : 1811256.6826883086, 7% - Money rerouted : 0, 0% 

What if 46606 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 26496 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 19551 is corrupted?
Money OK : 2229564.836584904, 8% - Money lost : 24395267.566751037, 92% - Money rerouted : 0, 0% 

What if 53667 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 45102 is corrupted?
Money OK : 263

## All ASes in the graph

In [19]:
for c in ripple_graph.nodes:
    if (c not in ripple_ases):
        corrupted_graph = ripple_graph.copy()
        corrupted_graph.remove_node(c)
        nx.set_edge_attributes(corrupted_graph, 0, 'weight')
        print('What if {} is corrupted?'.format(c))
        amount_ok, amount_lost, amount_rerouted = replay_transactions(transactions, corrupted_graph, ripple_graph)
        total = amount_ok + amount_lost + amount_rerouted
        percent_ok = amount_ok / total
        percent_ko = amount_lost / total
        percent_rerouted = amount_rerouted / total
        print('Money OK : {}, {:.0%} - Money lost : {}, {:.0%} - Money rerouted : {}, {:.0%} \n'.format(amount_ok, percent_ok, amount_lost, percent_ko, amount_rerouted, percent_rerouted))
        #analyse_graph(ripple_graph, ripple_ases)

What if 7506 is corrupted?
Money OK : 26624832.403263155, 100% - Money lost : 7.283394449113598e-05, 0% - Money rerouted : 0, 0% 

What if 2914 is corrupted?
Money OK : 22470118.0486835, 84% - Money lost : 0, 0% - Money rerouted : 4154714.3546524355, 16% 

What if 4637 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 6939 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 8220 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 3356 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 34779 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 41828 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 24482 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 

Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 6762 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 62785 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 4844 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 174 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 20013 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 2603 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 3267 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 4181 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 5769 is corrupted?
Money OK 

Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 14840 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 14907 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 15547 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 15576 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 15605 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 20764 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 20811 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 21320 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 22356 is corrupted?
M

Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 4651 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 4775 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 6619 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 6648 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 9050 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 9286 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 9381 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 12576 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 12713 is corrupted?
Money OK

Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 51873 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 53067 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 59891 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 64463 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 198290 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 263651 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 263674 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 262589 is corrupted?
Money OK : 26624832.40333599, 100% - Money lost : 0, 0% - Money rerouted : 0, 0% 

What if 18403 is corrupte